# Redes Neuronales Multicapa (MLP)

Una red MLP consiste en:
- Una capa de entrada
- Una o más capas ocultas con activaciones no lineales
- Una capa de salida con softmax o activación lineal según el caso

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Preprocesamiento del dataset original
df = pd.read_csv('Historia_Climatica.csv')
df["Fecha_Hora"] = pd.to_datetime(df["Fecha_Hora"], format="%m/%d/%y %H:%M")
df["Presion_hPa"] = df["Presion_hPa"] / 10

# Convertir clases a números
le = LabelEncoder()
df["Estado_Clima_Cod"] = le.fit_transform(df["Estado_Clima"])

# Selección de características y etiquetas
X = df[["Temperatura_C", "Humedad_%", "Vel_Viento_mps", "Presion_hPa"]].values
y = df["Estado_Clima_Cod"].values

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print("Datos cargados y procesados correctamente.")

Datos cargados y procesados correctamente.


## Capas ocultas y neuronas (1 capa oculta, 16 neuronas ocultas)

El número de capas ocultas y neuronas determina la **capacidad de representación** del modelo. Más capas → mayor complejidad, pero mayor riesgo de sobreajuste.

In [20]:
def relu(x):
  return np.maximum(0, x)

def softmax(x):
  e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
  return e_x / np.sum(e_x, axis=1, keepdims=True)

epochs = 200

class MLP:
  def __init__(self, input_dim, hidden_dim, output_dim, lr=0.01):
    self.W1 = np.random.randn(input_dim, hidden_dim) * np.sqrt(2. / input_dim)
    self.b1 = np.zeros((1, hidden_dim))
    self.W2 = np.random.randn(hidden_dim, output_dim) * np.sqrt(2. / hidden_dim)
    self.b2 = np.zeros((1, output_dim))
    self.lr = lr

  def forward(self, X):
    self.z1 = np.dot(X, self.W1) + self.b1
    self.a1 = relu(self.z1)
    self.z2 = np.dot(self.a1, self.W2) + self.b2
    self.a2 = softmax(self.z2)
    return self.a2

  def backward(self, X, y_true):
    m = y_true.shape[0]
    grad_output = self.a2.copy()
    grad_output[range(m), y_true] -= 1
    grad_output /= m

    dW2 = np.dot(self.a1.T, grad_output)
    db2 = np.sum(grad_output, axis=0, keepdims=True)

    da1 = np.dot(grad_output, self.W2.T)
    dz1 = da1 * (self.z1 > 0)

    dW1 = np.dot(X.T, dz1)
    db1 = np.sum(dz1, axis=0, keepdims=True)

    self.W2 -= self.lr * dW2
    self.b2 -= self.lr * db2
    self.W1 -= self.lr * dW1
    self.b1 -= self.lr * db1
  
  def train(self, X, y, epochs):
    for epoch in range(epochs):
      y_pred = self.forward(X)
      loss = -np.sum(np.log(y_pred[range(len(y)), y])) / len(y)
      acc = np.mean(np.argmax(y_pred, axis=1) == y)
      self.backward(X, y)
      if epoch % 10 == 0:
        print(f"Época {epoch}: Pérdida={loss:.4f}, Precisión={acc:.4f}")


In [24]:
# Entrenar MLP
mlp = MLP(input_dim=4, hidden_dim=16, output_dim=len(np.unique(y)), lr=0.1)
mlp.train(X_train, y_train, epochs)

# Evaluación
y_pred_test = mlp.forward(X_test)
test_accuracy = np.mean(np.argmax(y_pred_test, axis=1) == y_test)
print(f"Precisión final en test: {test_accuracy:.4f}")


Época 0: Pérdida=2.4600, Precisión=0.1638
Época 10: Pérdida=2.1085, Precisión=0.1799
Época 20: Pérdida=1.9695, Precisión=0.1774
Época 30: Pérdida=1.9014, Precisión=0.1811
Época 40: Pérdida=1.8646, Precisión=0.1811
Época 50: Pérdida=1.8425, Precisión=0.1762
Época 60: Pérdida=1.8280, Precisión=0.1873
Época 70: Pérdida=1.8175, Precisión=0.1886
Época 80: Pérdida=1.8097, Precisión=0.1935
Época 90: Pérdida=1.8034, Precisión=0.1898
Época 100: Pérdida=1.7983, Precisión=0.1911
Época 110: Pérdida=1.7939, Precisión=0.2010
Época 120: Pérdida=1.7901, Precisión=0.1985
Época 130: Pérdida=1.7869, Precisión=0.2047
Época 140: Pérdida=1.7842, Precisión=0.2060
Época 150: Pérdida=1.7818, Precisión=0.2047
Época 160: Pérdida=1.7796, Precisión=0.2097
Época 170: Pérdida=1.7777, Precisión=0.2109
Época 180: Pérdida=1.7759, Precisión=0.2097
Época 190: Pérdida=1.7743, Precisión=0.2159
Precisión final en test: 0.1535


## Versión extendida del MLP con múltiples capas ocultas (flexible y escalable)

In [31]:
from typing import List

# Funciones de activación y sus derivadas
def relu(x):
  return np.maximum(0, x)

def relu_derivada(x):
  return (x > 0).astype(float)

def softmax(x):
  e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
  return e_x / np.sum(e_x, axis=1, keepdims=True)

class MLP_Flexible:
  def __init__(self, capas: List[int], lr=0.01):
    """
    capas: lista con el número de neuronas por capa (incluyendo entrada y salida)
      Ej: [4, 32, 16, 6] → 2 capas ocultas
    """
    self.capas = capas
    self.lr = lr
    self.W = []
    self.b = []

    for i in range(len(capas) - 1):
      W_i = np.random.randn(capas[i], capas[i+1]) * np.sqrt(2 / capas[i])
      b_i = np.zeros((1, capas[i+1]))
      self.W.append(W_i)
      self.b.append(b_i)

  def forward(self, X):
    self.a = [X]
    self.z = []

    for i in range(len(self.W) - 1):
      z_i = np.dot(self.a[-1], self.W[i]) + self.b[i]
      a_i = relu(z_i)
      self.z.append(z_i)
      self.a.append(a_i)

    # Última capa (softmax)
    z_final = np.dot(self.a[-1], self.W[-1]) + self.b[-1]
    a_final = softmax(z_final)
    self.z.append(z_final)
    self.a.append(a_final)
    return a_final

  def backward(self, y_true):
    m = y_true.shape[0]
    grad = self.a[-1].copy()
    grad[range(m), y_true] -= 1
    grad /= m

    for i in reversed(range(len(self.W))):
      dW = np.dot(self.a[i].T, grad)
      db = np.sum(grad, axis=0, keepdims=True)

      self.W[i] -= self.lr * dW
      self.b[i] -= self.lr * db

      if i != 0:
        grad = np.dot(grad, self.W[i].T) * relu_derivada(self.z[i-1])

  def train(self, X, y, epochs):
    for epoch in range(epochs):
      y_pred = self.forward(X)
      loss = -np.sum(np.log(y_pred[range(len(y)), y])) / len(y)
      acc = np.mean(np.argmax(y_pred, axis=1) == y)
      self.backward(y)
      if epoch % 10 == 0:
        print(f"Época {epoch}: Pérdida={loss:.4f}, Precisión={acc:.4f}")

In [34]:
mlp = MLP_Flexible(capas=[4, 32, 16, len(np.unique(y))], lr=0.1)
mlp.train(X_train, y_train, epochs)
y_pred = mlp.forward(X_test)
acc = np.mean(np.argmax(y_pred, axis=1) == y_test)
print(f"Precisión final en test: {acc:.4f}")

Época 0: Pérdida=2.3244, Precisión=0.1737
Época 10: Pérdida=1.8287, Precisión=0.1873
Época 20: Pérdida=1.8014, Precisión=0.1960
Época 30: Pérdida=1.7897, Precisión=0.1948
Época 40: Pérdida=1.7828, Precisión=0.2035
Época 50: Pérdida=1.7779, Precisión=0.2134
Época 60: Pérdida=1.7740, Precisión=0.2171
Época 70: Pérdida=1.7706, Precisión=0.2146
Época 80: Pérdida=1.7676, Precisión=0.2109
Época 90: Pérdida=1.7650, Precisión=0.2196
Época 100: Pérdida=1.7624, Precisión=0.2246
Época 110: Pérdida=1.7600, Precisión=0.2258
Época 120: Pérdida=1.7577, Precisión=0.2246
Época 130: Pérdida=1.7557, Precisión=0.2320
Época 140: Pérdida=1.7537, Precisión=0.2370
Época 150: Pérdida=1.7519, Precisión=0.2407
Época 160: Pérdida=1.7499, Precisión=0.2419
Época 170: Pérdida=1.7482, Precisión=0.2481
Época 180: Pérdida=1.7468, Precisión=0.2494
Época 190: Pérdida=1.7455, Precisión=0.2519
Precisión final en test: 0.1634
